In [72]:
import re
import pandas as pd 
import nltk
from nltk.corpus import stopwords , opinion_lexicon
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix



nltk.download('vader_lexicon')
nltk.download('opinion_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


True

In [41]:
import warnings
warnings.filterwarnings('ignore')

In [42]:
df = pd.read_csv("C:/Users/Lenovo/Desktop/ds_tools_api/reviews.csv")
df.head()

,asin,title,comment,rate,helpful_votes,review_date,is_verified_purchase
0,B0CL5KNB9M,PlayStation 5 Digital Edition Fortnite Cobalt...,I recently purchased the PlayStation 5 Pro Di...,5,6 people found this helpful,"Reviewed in the United States on December 7, 2024",True
1,B0CL5KNB9M,Fives No Jive,A longtime PlayStation fan I had been putting ...,5,551 people found this helpful,"Reviewed in the United States on June 8, 2024",True
2,B0CL5KNB9M,PlayStation5 Console Slim Review,The PlayStation5 Console Slim is a fantastic e...,5,94 people found this helpful,"Reviewed in the United States on November 15, ...",True
3,B0CL5KNB9M,Compact Sleek and Reliable,The PS5 Digital Slim has worked flawlessly Its...,5,NaN,"Reviewed in the United States on December 11, ...",True
4,B0CL5KNB9M,works great,turn it on clean shiny fast load the only thin...,5,19 people found this helpful,"Reviewed in the United States on November 29, ...",True


In [43]:
# clean reviews content
df['comment'] = df['comment'].apply(lambda comment: re.sub(r'<[^>]+>|\W+|\d+', ' ', str(comment)).lower())

# tokenize comments
df['words'] = df['comment'].apply(lambda comment: nltk.word_tokenize(comment))

# remove stop words
stop_words = set(stopwords.words('english'))
df['words'] = df['words'].apply(lambda words: [word for word in words if word not in stop_words])

# join words back into a string
df['comment'] = df['words'].apply(lambda words: ' '.join(words))
df = df.drop(columns=['words'])



df.head()

,asin,title,comment,rate,helpful_votes,review_date,is_verified_purchase
0,B0CL5KNB9M,PlayStation 5 Digital Edition Fortnite Cobalt...,recently purchased playstation pro digital edi...,5,6 people found this helpful,"Reviewed in the United States on December 7, 2024",True
1,B0CL5KNB9M,Fives No Jive,longtime playstation fan putting getting ps th...,5,551 people found this helpful,"Reviewed in the United States on June 8, 2024",True
2,B0CL5KNB9M,PlayStation5 Console Slim Review,playstation console slim fantastic evolution o...,5,94 people found this helpful,"Reviewed in the United States on November 15, ...",True
3,B0CL5KNB9M,Compact Sleek and Reliable,ps digital slim worked flawlessly compact desi...,5,NaN,"Reviewed in the United States on December 11, ...",True
4,B0CL5KNB9M,works great,turn clean shiny fast load thing astros playro...,5,19 people found this helpful,"Reviewed in the United States on November 29, ...",True


In [ ]:
# tfidf for unigram
tfidf = TfidfVectorizer(max_features=2000, ngram_range=(1, 2))
rfidf_sparse = tfidf.fit_transform(df['comment'])

feature_names = tfidf.get_feature_names_out()


positive_words = set(opinion_lexicon.positive())
negative_words = set(opinion_lexicon.negative())


positive_scores = []
negative_scores = []

for row in rfidf_sparse.toarray():
    positive_score = sum([row[feature_names.tolist().index(word)] for word in positive_words if word in feature_names])
    negative_score = sum([row[feature_names.tolist().index(word)] for word in negative_words if  word in feature_names])
    positive_scores.append(positive_score)
    negative_scores.append(negative_score)

df['positive_score'] = positive_scores
df['negative_score'] = negative_scores

df['sentiment'] = df['positive_score'] - df['negative_score']
df['sentiment'] = df['sentiment'].apply(lambda sent: 'positive' if sent > 0 else 'negative' if sent < 0 else 'neutral')

# df.drop(columns=['positive_score', 'negative_score'], inplace=True)

# df = df[df['rate'] == 3]



# with open('reviews_labeled.csv', 'w') as f:
#     f.write(df.to_csv(index=False))


df.head()

,asin,title,comment,rate,helpful_votes,review_date,is_verified_purchase,positive_score,negative_score,sentiment
0,B0CL5KNB9M,PlayStation 5 Digital Edition Fortnite Cobalt...,recently purchased playstation pro digital edi...,5,6 people found this helpful,"Reviewed in the United States on December 7, 2024",True,1.491977,0.091272,positive
1,B0CL5KNB9M,Fives No Jive,longtime playstation fan putting getting ps th...,5,551 people found this helpful,"Reviewed in the United States on June 8, 2024",True,0.566011,0.305461,positive
2,B0CL5KNB9M,PlayStation5 Console Slim Review,playstation console slim fantastic evolution o...,5,94 people found this helpful,"Reviewed in the United States on November 15, ...",True,0.845969,0.000000,positive
3,B0CL5KNB9M,Compact Sleek and Reliable,ps digital slim worked flawlessly compact desi...,5,NaN,"Reviewed in the United States on December 11, ...",True,1.229438,0.000000,positive
4,B0CL5KNB9M,works great,turn clean shiny fast load thing astros playro...,5,19 people found this helpful,"Reviewed in the United States on November 29, ...",True,0.560571,0.320713,positive


In [77]:
with open('reviews_labeled.csv', 'w', encoding='utf-8') as f:
    f.write(df.to_csv(index=False))

In [45]:
Y = df['sentiment']
X = df['comment']

X_train, X_test, y_train, y_test = train_test_split(rfidf_sparse, Y, test_size=0.25, random_state=42)

In [46]:
X_sentiment = df['comment']
y_sentiment = df['sentiment']

converter = TfidfVectorizer(max_features=5000)
sentiment_tfidf = converter.fit_transform(X_sentiment)


X_train, X_test, y_train, y_test = train_test_split(sentiment_tfidf, y_sentiment, test_size=0.2, random_state=42)

logistic_model = LogisticRegression()
naive_model = MultinomialNB()

logistic_model.fit(X_train, y_train)
naive_model.fit(X_train, y_train)

log_prd = logistic_model.predict(X_test)
nb_prd = naive_model.predict(X_test)
classification_report_log=classification_report(y_test, log_prd)
classification_report_naive=classification_report(y_test, nb_prd)

print("Logistic Regression Sentiment Classification Report:\n")
print(classification_report_log)
print("Naive Bayes Sentiment Classification Report:\n")
print(classification_report_naive)
confusion_mat_log = confusion_matrix(y_test,log_prd)
confusion_mat_naive = confusion_matrix(y_test,nb_prd)

Logistic Regression Sentiment Classification Report:

              precision    recall  f1-score   support

    negative       0.79      0.97      0.87       176
     neutral       0.95      0.62      0.75        88
    positive       0.94      0.89      0.92       146

    accuracy                           0.87       410
   macro avg       0.89      0.83      0.85       410
weighted avg       0.88      0.87      0.86       410

Naive Bayes Sentiment Classification Report:

              precision    recall  f1-score   support

    negative       0.72      0.95      0.82       176
     neutral       1.00      0.42      0.59        88
    positive       0.86      0.82      0.84       146

    accuracy                           0.79       410
   macro avg       0.86      0.73      0.75       410
weighted avg       0.83      0.79      0.78       410



In [ ]:
fig = px.imshow(confusion_mat_log,
                labels=dict(x="Predicted", y="Actual", color="Count"),
                x=['positive', 'neutral', 'negative'],
                y=['positive', 'neutral', 'negative'],
                color_continuous_scale=px.colors.qualitative.Set3,
                title='Confusion Matrix of logistic',
                width=800)

fig.update_traces(text=confusion_mat_log, texttemplate="%{z}")
fig.show()

In [70]:
fig = px.imshow(confusion_mat_naive,
                labels=dict(x="Predicted", y="Actual", color="Count"),
                x=['positive', 'neutral', 'negative'],
                y=['positive', 'neutral', 'negative'],
                color_continuous_scale=px.colors.qualitative.Set3,
                title='Confusion Matrix of Naive',
                width=800)

fig.update_traces(text=confusion_mat_naive, texttemplate="%{z}")
fig.show()

In [49]:
classifier = RandomForestClassifier()

classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

acccuracy = classifier.score(X_test, y_test)

acccuracy

0.8365853658536585

In [67]:
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

acccuracy = classifier.score(X_test, y_test)
print(f"Accuracy: {acccuracy}")

cm = confusion_matrix(y_test, y_pred, labels=['positive', 'neutral', 'negative'])

fig = px.imshow(cm,
                labels=dict(x="Predicted", y="Actual", color="Count"),
                x=['positive', 'neutral', 'negative'],
                y=['positive', 'neutral', 'negative'],
                color_continuous_scale=px.colors.qualitative.Set3,
                title='Confusion Matrix for Random Forest',
                width=800)

fig.update_traces(text=cm, texttemplate="%{z}")
fig.show()

Accuracy: 0.824390243902439


In [ ]:
X_rate = df['comment']
y_rate = df['rate']

X_rate_tfidf = converter.fit_transform(X_rate)

X_train2, X_test2, y_train2, y_test2 = train_test_split(X_rate_tfidf, y_rate, test_size=0.2, random_state=42)

svm_model = LinearSVC()
svm_model.fit(X_train2, y_train2)

svm_pred = svm_model.predict(X_test2)

print("SVM Rate Prediction Report:\n")
print(classification_report(y_test2, svm_pred))

verified_analysis = df.groupby('is_verified_purchase')['rate'].mean()
print("\nAverage Ratings for Verified vs Non-verified Purchases:\n")
print(verified_analysis)


fig = go.Figure(data=[
    go.Bar(
        x=verified_analysis.index,
        y=verified_analysis.values,
        marker_color=['#8DD3C7', '#BEBADA']
    ),
    
])

fig.update_layout(
    title='Average Ratings: Verified vs Non-verified Purchases',
    yaxis_title='Average Rating',
    showlegend=False,
    template='plotly_white',
    width=800
)

fig.show()

SVM Rate Prediction Report:

              precision    recall  f1-score   support

           1       0.75      0.85      0.80       199
           2       0.67      0.60      0.63       117
           3       0.75      0.38      0.50         8
           4       0.33      0.08      0.13        12
           5       0.90      0.88      0.89        74

    accuracy                           0.75       410
   macro avg       0.68      0.56      0.59       410
weighted avg       0.74      0.75      0.74       410


Average Ratings for Verified vs Non-verified Purchases:

is_verified_purchase
False    1.543210
True     2.081425
Name: rate, dtype: float64


In [51]:
df.head()

,asin,title,comment,rate,helpful_votes,review_date,is_verified_purchase,positive_score,negative_score,sentiment
0,B0CL5KNB9M,PlayStation 5 Digital Edition Fortnite Cobalt...,recently purchased playstation pro digital edi...,5,6 people found this helpful,"Reviewed in the United States on December 7, 2024",True,1.491977,0.091272,positive
1,B0CL5KNB9M,Fives No Jive,longtime playstation fan putting getting ps th...,5,551 people found this helpful,"Reviewed in the United States on June 8, 2024",True,0.566011,0.305461,positive
2,B0CL5KNB9M,PlayStation5 Console Slim Review,playstation console slim fantastic evolution o...,5,94 people found this helpful,"Reviewed in the United States on November 15, ...",True,0.845969,0.000000,positive
3,B0CL5KNB9M,Compact Sleek and Reliable,ps digital slim worked flawlessly compact desi...,5,NaN,"Reviewed in the United States on December 11, ...",True,1.229438,0.000000,positive
4,B0CL5KNB9M,works great,turn clean shiny fast load thing astros playro...,5,19 people found this helpful,"Reviewed in the United States on November 29, ...",True,0.560571,0.320713,positive


In [62]:
sentiment_counts = df['sentiment'].value_counts()

fig = px.bar(x=sentiment_counts.index, 
             y=sentiment_counts.values,
             title='Distribution of Sentiment Labels',
             labels={'x': 'Sentiment', 'y': 'Count'},
             color=sentiment_counts.index,
             color_discrete_sequence=px.colors.qualitative.Set3,
             width=800)

fig.update_layout(
    title_x=0.5,
    xaxis_tickangle=45,
)

fig.show()

In [53]:
def extract_date_from_text(text):
    date_pattern = r"(January|February|March|April|May|June|July|August|September|October|November|December)\s+(\d{1,2}),\s+(\d{4})"
    date =  re.search(date_pattern, text) 
    parsed_date = pd.to_datetime(date.group(0))
    return parsed_date

# Time series

In [54]:
df_time_series = df[['review_date','asin', 'positive_score', 'negative_score', 'sentiment']]

In [55]:

df_time_series['review_date'] = df_time_series['review_date'].apply(extract_date_from_text)

df_time_series = df_time_series.sort_values(by='review_date')
df_time_series.set_index('review_date', inplace=True)

In [74]:

month_df = df_time_series.resample('YE').agg({
    'positive_score': 'mean',
    'negative_score': 'mean',
    'sentiment': lambda x: (x == 'negative').sum()
})

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=month_df.index,
    y=month_df['sentiment'],
    name='Negative count',
    line=dict(color='red')
))


fig.update_layout(
    title='Negative Over Time',
    xaxis_title='Year',
    yaxis_title='Count',
    showlegend=True,
    width=800,
    height=400,
)



fig.show()